---
**Turma:** 15

**Grupo:** 11 - Bruno Correia, Gilson Costa, Vivian Liu

**Contato**: `brunovpm@hotmail.com, gilson.costa@gmail.com, liuoliveira.vivian@gmail.com`


**Licença deste notebook**:
![CC BY](https://licensebuttons.net/l/by/3.0/88x31.png)

[Clique aqui para saber mais sobre a licença CC BY v4.0](https://creativecommons.org/licenses/by/4.0/legalcode.pt)

---
# Regressão Logistica

In [0]:
root_dir = '/dbfs/FileStore/ifood'

dbutils.fs.ls(f'{root_dir}')

Out[3]: [FileInfo(path='dbfs:/dbfs/FileStore/ifood/abt/', name='abt/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/ifood/customer_segmentation/', name='customer_segmentation/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/ifood/enrich/', name='enrich/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/ifood/marketing_push_full/', name='marketing_push_full/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/ifood/orders/', name='orders/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/ifood/orders_with_cost_revenue/', name='orders_with_cost_revenue/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/ifood/preprocess/', name='preprocess/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/ifood/sessions_visits/', name='sessions_visits/', size=0)]

In [0]:
# df_orders = spark.read.parquet(f'{root_dir}/orders').repartition(2).cache()
# df_orders.createOrReplaceTempView('orders')

df_customer = spark.read.parquet(f'{root_dir}/customer_segmentation').repartition(2).cache()
df_customer.createOrReplaceTempView('customer')

df_push_session_nov = spark.read.parquet(f'{root_dir}/abt/push_session_nov').repartition(2).cache()
df_push_session_nov.createOrReplaceTempView('push_session_nov')

df_push_session_oct = spark.read.parquet(f'{root_dir}/abt/push_session_oct').repartition(2).cache()
df_push_session_oct.createOrReplaceTempView('push_session_oct')

df_push_session_oct = spark.read.parquet(f'{root_dir}/abt/push_session_aug').repartition(2).cache()
df_push_session_oct.createOrReplaceTempView('push_session_aug')

df_push_session_nov_half = spark.read.parquet(
  f'{root_dir}/abt/push_session_nov_half'
).repartition(2).cache()
df_push_session_nov_half.createOrReplaceTempView('push_session_nov_half')

df_push_session_oct_half = spark.read.parquet(
  f'{root_dir}/abt/push_session_oct_half'
).repartition(2).cache()
df_push_session_oct_half.createOrReplaceTempView('push_session_oct_half')

df_push_session_aug_half = spark.read.parquet(
  f'{root_dir}/abt/push_session_aug_half'
).repartition(2).cache()
df_push_session_aug_half.createOrReplaceTempView('push_session_aug_half')

# ABT dados do customer_segmentation

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW abt_case_customer_view AS 

SELECT
IF(ifood_status IN('Churn', 'Inactive'), 1, 0) label,
orders_last_91d,
qtt_orders_last_year,
qtt_valid_orders,
qtt_invalid_orders,
recency_months,
customer_lifetime_days,
customer_lifetime_months,
was_mub_last_month,
buyer_last_91d,
rfv_score,
recency_days,
freq_last_91d,
maturity_orders,
merchant_variety,
merchant_offer

FROM customer

WHERE segmentation_month = '2019-12-01'

# ABT dados de push e session mensal

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW abt_case_month_view AS 

SELECT
customer.orders_last_91d,
customer.qtt_orders_last_year,
customer.qtt_valid_orders,
customer.qtt_invalid_orders,
customer.recency_months,
customer.customer_lifetime_days,
customer.customer_lifetime_months,
customer.was_mub_last_month,
customer.buyer_last_91d,
customer.rfv_score,
customer.recency_days,
customer.freq_last_91d,
customer.maturity_orders,
customer.merchant_variety,
customer.merchant_offer,
push.total_bounce_dawn,
push.total_bounce_breakfast,
push.total_bounce_lunch,
push.total_bounce_snack,
push.total_bounce_dinner,
push.total_click_dawn,
push.total_click_breakfast,
push.total_click_lunch,
push.total_click_snack,
push.total_click_dinner,
push.total_send,
push.total_received,
push.total_bounce,
push.total_click,
push.sum_event_open,
push.sum_view_restaurant_screen,
push.sum_view_dish_screen,
push.sum_click_add_item,
push.sum_view_checkout,
push.sum_callback_purchase,
push.order_session_quantity,
IF(target.ifood_status_last_month IN('Churn', 'Inactive'), 1, 0) label_last_month,
IF(target.ifood_status IN('Churn', 'Inactive'), 1, 0) label,
target.customer_id

FROM customer target

LEFT JOIN push_session_nov push
ON push.customer_id = target.customer_id

LEFT JOIN customer customer
ON customer.customer_id = target.customer_id
AND customer.segmentation_month = '2019-11-01'

WHERE target.segmentation_month = '2019-12-01'

In [0]:
df_abt_month_view = spark.table('abt_case_month_view')
df_abt_month_view.count()

Out[58]: 30127

# Verificar duplicates e nulls

In [0]:
df_abt_month_view.summary().toPandas().transpose()

,0,1,2,3,4,5,6,7
summary,count,mean,stddev,min,25%,50%,75%,max
orders_last_91d,30127,8.703355793806221,10.292124794873173,1.0,2.0,5.0,11.0,186.0
qtt_orders_last_year,30127,25.047332957148072,32.20220013694818,1,5,15,32,718
qtt_valid_orders,30127,43.128124273907126,66.2698163274547,1,7,21,51,1228
qtt_invalid_orders,30127,1.325123643243602,2.4189196363688183,0,0,0,2,100
recency_months,30127,1.7988076281017418,2.0495283043812518,0.0,0.16129032,0.67741935,3.4516129,5.90322581
customer_lifetime_days,30127,645.1693497527135,509.58951839830894,152,223,461,947,2889
customer_lifetime_months,30127,20.698111328708467,16.74426137132443,4,7,15,31,94
was_mub_last_month,30127,0.5521293192153218,0.49728336223376907,0,0,1,1,1
buyer_last_91d,30127,0.7390380721611843,0.4391664853597274,0,0,1,1,1


# Missing and Duplicates

In [0]:
df_abt_month_final = df_abt_month_view.fillna(0).dropDuplicates()

In [0]:
df_abt_month_final.write.parquet(f'{root_dir}/abt/abt_case_month', mode='overwrite')

In [0]:
df_abt_case_month = spark.read.parquet(
  f'{root_dir}/abt/abt_case_month'
).repartition(2).cache()
df_abt_case_month.createOrReplaceTempView('abt_case_month')
df_abt_case_month.count()

Out[62]: 30079

In [0]:
df_abt_case_month_train, df_abt_case_month_test = df_abt_case_month.randomSplit([0.7, 0.3], seed=42)

In [0]:
df_abt_case_month_train.write.parquet(f'{root_dir}/abt/abt_case_month_train', mode='overwrite')
df_abt_case_month_test.write.parquet(f'{root_dir}/abt/abt_case_month_test', mode='overwrite')

# ABT dados de push e session (15 dias)

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW abt_case_half_view AS 

SELECT
-- orders_last_91d,
-- qtt_orders_last_year,
-- qtt_valid_orders,
-- qtt_invalid_orders,
-- recency_months,
-- customer_lifetime_days,
-- customer_lifetime_months,
-- was_mub_last_month,
-- buyer_last_91d,
-- rfv_score,
-- recency_days,
-- freq_last_91d,
-- maturity_orders,
-- merchant_variety,
-- merchant_offer,
push.total_bounce_dawn,
push.total_bounce_breakfast,
push.total_bounce_lunch,
push.total_bounce_snack,
push.total_bounce_dinner,
push.total_click_dawn,
push.total_click_breakfast,
push.total_click_lunch,
push.total_click_snack,
push.total_click_dinner,
push.total_send,
push.total_bounce,
push.total_click,
push.sum_event_open,
push.sum_view_restaurant_screen,
push.sum_view_dish_screen,
push.sum_click_add_item,
push.sum_view_checkout,
push.sum_callback_purchase,
push.order_session_quantity,

IF(target.ifood_status IN('Churn', 'Inactive'), 1, 0) label,
target.customer_id

FROM customer target

LEFT JOIN push_session_november_half push
ON push.customer_id = target.customer_id

WHERE target.segmentation_month = '2019-12-01'

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Table or view not found: push_session_november_half; line 31 pos 10;
'Project ['push.total_bounce_dawn, 'push.total_bounce_breakfast, 'push.total_bounce_lunch, 'push.total_bounce_snack, 'push.total_bounce_dinner, 'push.total_click_dawn, 'push.total_click_breakfast, 'push.total_click_lunch, 'push.total_click_snack, 'push.total_click_dinner, 'push.total_send, 'push.total_bounce, 'push.total_click, 'push.sum_event_open, 'push.sum_view_restaurant_screen, 'push.sum_view_dish_screen, 'push.sum_click_add_item, 'push.sum_view_checkout, 'push.sum_callback_purchase, 'push.order_session_quantity, 'IF('customer.ifood_status IN (Churn,Inactive), 1, 0) AS label_last_month#1668, 'IF('target.ifood_status IN (Churn,Inactive), 1, 0) AS label#1669, 'target.customer_id]
+- 'Filter ('target.segmentation_month = 2019-12-01)
 +- 'Join LeftOuter, ('push.customer_id = 'target.customer_id)
 :- SubqueryAlias target
 : +- SubqueryAlias customer
 : +- Repartition 2, true
 : +- Relation[customer_id#43,segmentation_month#44,ifood_status_last_month#45,ifood_status#46,orders_last_91d#47,qtt_orders_last_year#48L,qtt_valid_orders#49L,last_valid_order_date#50,qtt_invalid_orders#51L,last_invalid_order_date#52,marlin_tag#53,recency_months#54,last_nps#55,registration_date#56,customer_lifetime_days#57L,customer_lifetime_months#58L,top_3_merchants_code#59,was_mub_last_month#60L,buyer_last_91d#61L,top_city#62,top_district#63,top_centroid_id#64,first_order_date#65,last_order_date#66,... 26 more fields] parquet
 +- 'SubqueryAlias push
 +- 'UnresolvedRelation [push_session_november_half]

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1(CheckAnalysis.scala:109)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1$adapted(CheckAnalysis.scala:95)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:184)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:183)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:183)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:183)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:183)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:183)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:183)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:183)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:183)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:183)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:183)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:183)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:183)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis(CheckAnalysis.scala:95)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis$(CheckAnalysis.scala:92)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:136)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:164)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:201)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:161)
	at org.apache.spark.sql.execution.Q

In [0]:
df_abt_half_view = spark.table('abt_case_half_view')
df_abt_half_view.count()

Out[38]: 30127

# Verificar duplicates e nulls

In [0]:
df_abt_half_view.summary().toPandas().transpose()

,0,1,2,3,4,5,6,7
summary,count,mean,stddev,min,25%,50%,75%,max
total_bounce_dawn,23102,1.2985888667647823E-4,0.011395071038371103,0,0,0,0,1
total_bounce_breakfast,23102,0.0031166132802354775,0.05949719478549834,0,0,0,0,2
total_bounce_lunch,23102,0.013591896805471388,0.12651097849036236,0,0,0,0,4
total_bounce_snack,23102,0.0020777421868236516,0.04553586420623652,0,0,0,0,1
total_bounce_dinner,23102,0.015929356765647995,0.1429614967570223,0,0,0,0,5
total_click_dawn,23102,0.005670504718206216,0.08118456684526767,0,0,0,0,2
total_click_breakfast,23102,0.03333044758029608,0.23761796733637836,0,0,0,0,6
total_click_lunch,23102,0.23002337459960176,0.7693552672636039,0,0,0,0,12
total_click_snack,23102,0.06527573370270973,0.3777021854494726,0,0,0,0,12


# Missing and Duplicates

In [0]:
df_abt_half_final = df_abt_half_view.fillna(0).dropDuplicates()

In [0]:
df_abt_half_final.write.parquet(f'{root_dir}/abt/abt_case_half', mode='overwrite')

In [0]:
df_abt_case_half = spark.read.parquet(
  f'{root_dir}/abt/abt_case_half'
).repartition(2).cache()
df_abt_case_half.createOrReplaceTempView('abt_case_half')
df_abt_case_half.count()

Out[42]: 30079

In [0]:
df_abt_case_half_train, df_abt_case_half_test = df_abt_case_half.randomSplit([0.7, 0.3], seed=42)

In [0]:
df_abt_case_half_train.write.parquet(f'{root_dir}/abt/abt_case_half_train', mode='overwrite')
df_abt_case_half_test.write.parquet(f'{root_dir}/abt/abt_case_half_test', mode='overwrite')

# ABT dados de push e session (15 dias final do mês)

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW abt_case_half_end_view AS 

SELECT
-- customer.orders_last_91d,
-- customer.qtt_orders_last_year,
-- customer.qtt_valid_orders,
-- customer.qtt_invalid_orders,
-- customer.recency_months,
-- customer.customer_lifetime_days,
-- customer.customer_lifetime_months,
-- customer.was_mub_last_month,
-- customer.buyer_last_91d,
-- customer.rfv_score,
-- customer.recency_days,
-- customer.freq_last_91d,
-- customer.maturity_orders,
-- customer.merchant_variety,
-- customer.merchant_offer,

push.total_bounce_dawn,
push.total_bounce_breakfast,
push.total_bounce_lunch,
push.total_bounce_snack,
push.total_bounce_dinner,

push.total_click_dawn,
push.total_click_breakfast,
push.total_click_lunch,
push.total_click_snack,
push.total_click_dinner,

push.total_send,
push.total_received,
push.total_bounce,
push.total_click,

push.sum_event_open,
push.sum_view_restaurant_screen,
push.sum_view_dish_screen,
push.sum_click_add_item,
push.sum_view_checkout,
push.sum_callback_purchase,
push.order_session_quantity,

IF(ifood_status IN('Churn', 'Inactive'), 1, 0) label,
target.customer_id

FROM customer target

LEFT JOIN push_session_november_half push
ON push.customer_id = target.customer_id

WHERE target.segmentation_month = '2019-11-01'

In [0]:
df_abt_half_end_view = spark.table('abt_case_half_end_view')

### Verificar duplicates e nulls

In [0]:
df_abt_half_end_view.summary().toPandas().transpose()

,0,1,2,3,4,5,6,7
summary,count,mean,stddev,min,25%,50%,75%,max
total_bounce_dawn,23168,1.294889502762431E-4,0.011378829972141522,0,0,0,0,1
total_bounce_breakfast,23168,0.003107734806629834,0.05941261637481121,0,0,0,0,2
total_bounce_lunch,23168,0.01355317679558011,0.1263327197489724,0,0,0,0,4
total_bounce_snack,23168,0.0020718232044198894,0.045471089690892724,0,0,0,0,1
total_bounce_dinner,23168,0.015883977900552487,0.14276023608112787,0,0,0,0,5
total_click_dawn,23168,0.005654350828729282,0.08106940519484111,0,0,0,0,2
total_click_breakfast,23168,0.03332182320441989,0.23745564590832066,0,0,0,0,6
total_click_lunch,23168,0.23014502762430938,0.7690787599036933,0,0,0,0,12
total_click_snack,23168,0.06543508287292818,0.3789470007601332,0,0,0,0,12


### Missing and Duplicates
_ Eliminado registros duplicados e preenchimento com zero para os valores nulls_

In [0]:
df_abt_half_end_final = df_abt_half_end_view.fillna(0).dropDuplicates()

In [0]:
df_abt_half_end_final.write.parquet(f'{root_dir}/abt/abt_case_half_end', mode='overwrite')

In [0]:
df_abt_case_half_end = spark.read.parquet(
  f'{root_dir}/abt/abt_case_half_end'
).repartition(2).cache()
df_abt_case_half_end.createOrReplaceTempView('abt_case_half_end')
df_abt_case_half_end.count()

Out[28]: 30079

In [0]:
df_abt_case_half_end_train, df_abt_case_half_end_test = df_abt_case_half_end.randomSplit([0.7, 0.3], seed=42)

In [0]:
df_abt_case_half_end_train.write.parquet(f'{root_dir}/abt/abt_case_half_end_train', mode='overwrite')
df_abt_case_half_end_test.write.parquet(f'{root_dir}/abt/abt_case_half_end_test', mode='overwrite')

# ABT dados de push e session (15 dias final do mês)

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW abt_case_aug_half_end_view AS 

SELECT
customer.orders_last_91d,
customer.qtt_orders_last_year,
customer.qtt_valid_orders,
customer.qtt_invalid_orders,
customer.recency_months,
customer.customer_lifetime_days,
customer.customer_lifetime_months,
customer.was_mub_last_month,
customer.buyer_last_91d,
customer.rfv_score,
customer.recency_days,
customer.freq_last_91d,
customer.maturity_orders,
customer.merchant_variety,
customer.merchant_offer,

push.total_send as total_send,
push.total_received as total_received,
push.total_bounce as total_bounce,
push.total_click as total_click,

push.sum_event_open as sum_event_open,
push.sum_view_restaurant_screen as sum_view_restaurant_screen,
push.sum_view_dish_screen as sum_view_dish_screen,
push.sum_click_add_item as sum_click_add_item,
push.sum_view_checkout as sum_view_checkout,
push.sum_callback_purchase as sum_callback_purchase,
push.order_session_quantity as order_session_quantity,

target.customer_id,
IF(target.ifood_status_last_month IN('Churn', 'Inactive'), 1, 0) label_last_month,
IF(target.ifood_status IN('Churn', 'Inactive'), 1, 0) label

FROM customer target

LEFT JOIN push_session_aug_half push
ON push.customer_id = target.customer_id

INNER JOIN customer
ON target.customer_id = customer.customer_id
AND customer.segmentation_month = '2019-07-01'

WHERE target.segmentation_month = '2019-08-01'

In [0]:
df_abt_case_aug_half_end_view = spark.table('abt_case_aug_half_end_view')

### Verificar duplicates e nulls

In [0]:
df_abt_case_aug_half_end_view.summary().toPandas().transpose()

,0,1,2,3,4,5,6,7
summary,count,mean,stddev,min,25%,50%,75%,max
orders_last_91d,30079,8.430466438378936,9.706358184540264,1.0,2.0,5.0,11.0,170.0
qtt_orders_last_year,30079,20.60078460055188,28.66795063758494,1,4,11,26,690
qtt_valid_orders,30079,34.22610459124306,57.60856043341954,1,4,14,39,1016
qtt_invalid_orders,30079,0.9308487649190466,1.9469164312743634,0,0,0,1,93
recency_months,30079,0.7432781843545332,0.6062911637457079,0.0,0.16129032,0.61290323,1.25806452,1.93548387
customer_lifetime_days,30079,527.1702849163869,511.8506013421399,30,102,345,831,2767
customer_lifetime_months,30079,16.86465640480069,16.815564699965726,1,3,11,27,90
was_mub_last_month,30079,1.0,0.0,1,1,1,1,1
buyer_last_91d,30079,1.0,0.0,1,1,1,1,1


### Missing and Duplicates
_ Eliminado registros duplicados e preenchimento com zero para os valores nulls_

In [0]:
df_abt_case_aug_half_end_final = df_abt_case_aug_half_end_view.fillna(0).dropDuplicates()

In [0]:
df_abt_case_aug_half_end_final.write.parquet(f'{root_dir}/abt/abt_case_aug_half_end', mode='overwrite')

In [0]:
df_abt_case_aug_half_end = spark.read.parquet(
  f'{root_dir}/abt/abt_case_aug_half_end'
).repartition(2).cache()
df_abt_case_aug_half_end.createOrReplaceTempView('abt_case_aug_half_end')

In [0]:
df_abt_case_aug_half_end_train, df_abt_case_aug_half_end_test = df_abt_case_aug_half_end.randomSplit(
  [0.7, 0.3],
  seed=42
)

In [0]:
df_abt_case_aug_half_end_train.write.parquet(
  f'{root_dir}/abt/abt_case_aug_half_end_train',
  mode='overwrite'
)

df_abt_case_aug_half_end_test.write.parquet(
  f'{root_dir}/abt/abt_case_aug_half_end_test',
  mode='overwrite'
)